In [1]:

#Montando local para alimentacão das bases do twitter e de cidades brasileiras
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Instalação e importação das bibliotecas necessarias
!pip install nltk
!pip install fuzzywuzzy
!pip install rapidfuzz
!pip install jsonmerge
!pip install xlrd
import pandas as pd
import difflib as dif
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import re
from jsonmerge import merge
from pandas.io.json import json_normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, WordPunctTokenizer
ambiguas = ['Água Boa','Brasil','brasil','br','Água Branca', 'Alagoinha', 'Alto Alegre', 'Alto Paraíso', 'Alvorada', 'Amparo', 'Anchieta', 'Antônio Carlos', 'Aparecida', 'Araguanã', 'Araruna', 'Areia Branca', 'Atalaia', 'Aurora', 'Bandeirantes', 'Baraúna', 'Barra Bonita', 'Barracão', 'Barro Alto', 'Batalha', 'Belém', 'Belmonte', 'Boa Esperança', 'Boa Vista', 'Bocaina', 'Bom Jardim', 'Bom Jesus', 'Bom Jesus do Tocantins', 'Bom Sucesso', 'Bonfim', 'Bonito', 'Borborema', 'Brejinho', 'Buritis', 'Cachoeira Dourada', 'Cachoeirinha', 'Cafelândia', 'Caiçara', 'Campestre', 'Campo Alegre', 'Campo Grande', 'Canápolis', 'Canarana', 'Candeias', 'Cantagalo', 'Capanema', 'Capela', 'Caracol', 'Caraúbas', 'Cascavel', 'Catanduvas', 'Cedral', 'Cedro', 'Centenário', 'Colinas', 'Colorado', 'Condado', 'Conde', 'Cruzeiro do Sul', 'Davinopólis', 'Douradinha', 'Eldorado', 'Entre Rios', 'Esperantina', 'Estrela do Norte', 'Fátima', 'Feira Nova', 'Filadélfia', 'Floresta', 'Formoso', 'General Carneiro', 'Gravataí', 'Guaíra', 'Guaraci', 'Guaraciaba', 'Hidrolândia', 'Humaitá', 'Iguatu', 'Inajá', 'Independência', 'Indianápolis', 'Ipueiras', 'Iracema', 'Irati', 'Itabaiana', 'Itajá', 'Itambé', 'Itapeva', 'Itapiranga', 'Itaporanga', 'Jaborandi', 'Jacutinga', 'Jandaíra', 'Japurá', 'Jardim', 'Jardinópolis', 'Jatobá', 'Jundiá', 'Jurema', 'Jussara', 'Lagoa Grande', 'Lagoa Santa', 'Lajeado', 'Laranjal', 'Maravilha', 'Massaranduba', 'Mesquita', 'Milagres', 'Mirador', 'Monte Alegre', 'Monte Castelo', 'Morrinhos', 'Mulungu', 'Mundo Novo', 'Natividade', 'Nazaré', 'Nova Aurora', 'Nova Fátima', 'Nova Olímpia', 'Nova Olinda', 'Nova Santa Rita', 'Nova União', 'Nova Veneza', 'Novo Horizonte', 'Novo Santo Antônio', 'Ouro Branco', 'Ouro Verde', 'Pacatuba', 'Palestina', 'Palmas', 'Palmeira', 'Palmital', 'Paraíso', 'Parnamirim', 'Passagem', 'Pau D’Arco', 'Paulista', 'Pedra Branca', 'Pedra Preta', 'Petrolândia', 'Pilar', 'Pilões', 'Pinhalzinho', 'Pinhão', 'Pitangueiras', 'Planaltina', 'Planalto', 'Praia Grande', 'Prata', 'Presidente Bernardes', 'Presidente Dutra', 'Presidente Juscelino', 'Presidente Kennedy', 'Presidente Médici', 'Primavera', 'Queimadas', 'Redenção', 'Riachão', 'Riachinho', 'Riacho de Santana', 'Riachuelo', 'Rio Branco', 'Rio Claro', 'Rio Negro', 'Rio Verde', 'Ruy Barbosa', 'Salgadinho', 'Saltinho', 'Santa Bárbara', 'Santa Cecília', 'Santa Cruz', 'Santa Filomena', 'Santa Helena', 'Santa Inês', 'Santa Isabel', 'Santa Lúcia', 'Santa Luzia', 'Santa Maria', 'Santa Rita', 'Santa Rosa de Lima', 'Santa Teresinha', 'Santa Terezinha', 'Santana', 'Santarém', 'Santo André', 'São Bento', 'São Carlos', 'São Domingos', 'São Francisco', 'São Francisco de Paula', 'São Gabriel', 'São Gonçalo do Amarante', 'São João', 'São João Batista', 'São João do Paraíso', 'São José do Divino', 'São Martinho', 'São Pedro', 'São Sebastião', 'São Simão', 'São Tomé', 'São Vicente', 'São Vicente Ferrer', 'Sapucaia', 'Sarandi', 'Serrinha', 'Sertãozinho', 'Sítio Novo', 'Sobradinho', 'Soledade', 'Tabatinga', 'Tangará', 'Tapejara', 'Tapira', 'Tapiraí', 'Tavares', 'Teodoro Sampaio', 'Terra Nova', 'Terra Roxa', 'Toledo', 'Trindade', 'Triunfo', 'Turmalina', 'Turvo', 'Valença', 'Vargem', 'Vargem Bonita', 'Várzea', 'Várzea Grande', 'Vera Cruz', 'Viana', 'Viçosa', 'Wenceslau Braz', 'São Paulo', 'Rio de Janeiro','são paulo']
ambiguas.sort()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 23.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for jsonmerge: filename=jsonmerge-1.9.0-py3-none-any.whl size=18633 sha256=4e9419335310f7b2bf815d71df02c6196764291eb1de822e3f44457cb954e75b
  Stored in directory: /root/.cache/pip/wheels/82/c3/9a/421a7eabc04d18b19ee10e43646bea935e518eb88a6e5e9df7
Successfully built jsonmerge
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.8/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#Abertura da base de tuítes, normalização dos niveis
with open('/content/drive/MyDrive/TCC/SRC/dataset_2021-04-27.json') as json_file:  
    d = json.load(json_file)
    rec = d
    dataset = pd.json_normalize(rec['tweets'], max_level=6)
dataset.to_csv('sample.csv')

In [4]:
import glob

json_dir = '/content/drive/MyDrive/TCC/SRC'

json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read())['tweets'],max_level=6)
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
df = pd.concat(dfs)

In [6]:
print(df.head(1))

                       created_at                   id               id_str  \
0  Wed Apr 28 23:59:31 +0000 2021  1387557117951975428  1387557117951975428   

                                           full_text  truncated  \
0  Atualização dos dados do Brasil. #coronavirus ...      False   

  display_text_range                                             source  \
0            [0, 54]  <a href="https://ifttt.com" rel="nofollow">IFT...   

   in_reply_to_status_id in_reply_to_status_id_str  in_reply_to_user_id  ...  \
0                    NaN                      None                  NaN  ...   

  quoted_status.place.name quoted_status.place.full_name  \
0                      NaN                           NaN   

   quoted_status.place.country_code  quoted_status.place.country  \
0                               NaN                          NaN   

   quoted_status.place.contained_within quoted_status.place.bounding_box.type  \
0                                   NaN                

In [ ]:
#Atribuição a um data frame - uma base
df = pd.concat([df['id'],df['user.location']],axis=1,keys=['id', 'localizacao'])
df.astype({"localizacao": str})
df = df.convert_dtypes()
df.dtypes

In [ ]:
#df = re.sub('\W+',' ',))

In [ ]:
#Atribuição a um data frame - uma base
#df = pd.concat([dataset['id'],dataset['user.location']],axis=1,keys=['id', 'localizacao'])

In [ ]:
base_municipios = pd.read_excel(r'/content/drive/MyDrive/TCC/SRC/RELATORIO_DTB_BRASIL_MUNICIPIO.xlsx')

In [ ]:
#União das bases de municípios e estados brasileiros
in_first = set(list(base_municipios.Nome_Município))
in_second = set(list(base_municipios.Nome_UF))
in_second_but_not_in_first = in_second - in_first
result = list(base_municipios.Nome_Município) + list(in_second_but_not_in_first)
for i in range(len(result)):

    result[i] = result[i].lower()

In [ ]:
dic = df.set_index('id').T.to_dict('records')
print(dic)

In [ ]:
tweet_tokenizer = TweetTokenizer()
wpt = WordPunctTokenizer()
import nltk
nltk.download('punkt')
nltk.download('stopwords')
portuguesestopwords = set(stopwords.words('portuguese'))
#portuguesestopwords

In [ ]:
def fillVmatch(x):
    word_to_check = x
    if not word_to_check:
        return 0
    a = process.extract(word_to_check,result,limit=1)
    if a[0][1]>= 90 and a[0][1]<= 100:
      # print('----FILLVMATCH----' + x + ' ' + str(a[0][1]))
         return a[0][1]
    else:
         return 0
def fillVName(c):
    word_to_check = c
    if not word_to_check:
      return 0
    a = process.extract(word_to_check,result, limit=1)
    if a[0][1]>= 90 and a[0][1]<= 100:
      # print('----FILLVNAME----' + c + ' ' + str(a[0][0]))
      return a[0][0]
    else:
        return ''

In [ ]:
def lower_dict(d):
   new_dict = dict((k, v.lower()) for k, v in d.items())
   return new_dict

In [ ]:
for i in range(len(dic)):
    dic[i] = lower_dict(dic[i])
str_list = list(filter(None, dic))
print(str_list)


In [ ]:
# print(dic[0])
dicionario = dic[0]
dicionario2 = {}
for i in dicionario:
    #dicionario[i] = word_tokenize(dicionario[i])
    #dicionario[i] = [x for x in dicionario[i] if x not in portuguesestopwords]
    listaTemp = []
    #print(dicionario[i])
   # for value in dicionario[i]:
       #print(teste)
    sublista = []

    sublista.append(dicionario[i])
    sublista.append(fillVmatch(dicionario[i]))
    sublista.append(fillVName(dicionario[i]))

    listaTemp.append(sublista)

    dicionario2[i] = listaTemp
    print(dicionario2)
    # tokens_without_stopwords = [x for x in tokens if x not in portuguesestopwords]   
    # print(tokens_without_stopwords)
    # print(fillVmatch(str(tokens_without_stopwords)))
    # print(fillVName(str(tokens_without_stopwords)))
    #a.append(tokens_without_stopwords)
print(dicionario2)
#fdist = nltk.FreqDist(p.lower() for p in a)


In [ ]:
dic[0]

In [ ]:
dicionario = dic[0]
dicionario2 = {}
index = 0
limit = 15
df2 = pd.DataFrame(columns=('ID', '%Similaridade', 'Local_Sugerido','Ambiguidade'))
for i in dicionario:
   index += 1
   if index == limit:
        break
   res = str(fillVName(re.sub('\W+',' ', dicionario[i])))
   if not dif.get_close_matches(res, ambiguas, n=1, cutoff=0.8):
    df2.loc[i] = [str(re.sub('\W+',' ', dicionario[i])), str(fillVmatch(dicionario[i])), res,'N']
   else:
    df2.loc[i] = [str(re.sub('\W+',' ', dicionario[i])), str(fillVmatch(dicionario[i])), res,'S']
index = df2.index
df2['ID_correto'] = index
df2 = df2.astype({"ID_correto": str}, errors='raise') 
df2.to_excel("teste4.xlsx", sheet_name='Sheet_name_1')

In [ ]:
index = df2.index
df2['ID_correto'] = index
df2 = df2.astype({"ID_correto": str}, errors='raise') 
df2.to_excel("86-95Teste7.xlsx", sheet_name='Sheet_name_1')

In [ ]:
##dicionario = dic[0]
#word_to_check = 'Esperantina'
#ambiguas = ['Água Boa', 'Água Branca', 'Alagoinha', 'Alto Alegre', 'Alto Paraíso', 'Alvorada', 'Amparo', 'Anchieta', 'Antônio Carlos', 'Aparecida', 'Araguanã', 'Araruna', 'Areia Branca', 'Atalaia', 'Aurora', 'Bandeirantes', 'Baraúna', 'Barra Bonita', 'Barracão', 'Barro Alto', 'Batalha', 'Belém', 'Belmonte', 'Boa Esperança', 'Boa Vista', 'Bocaina', 'Bom Jardim', 'Bom Jesus', 'Bom Jesus do Tocantins', 'Bom Sucesso', 'Bonfim', 'Bonito', 'Borborema', 'Brejinho', 'Buritis', 'Cachoeira Dourada', 'Cachoeirinha', 'Cafelândia', 'Caiçara', 'Campestre', 'Campo Alegre', 'Campo Grande', 'Canápolis', 'Canarana', 'Candeias', 'Cantagalo', 'Capanema', 'Capela', 'Caracol', 'Caraúbas', 'Cascavel', 'Catanduvas', 'Cedral', 'Cedro', 'Centenário', 'Colinas', 'Colorado', 'Condado', 'Conde', 'Cruzeiro do Sul', 'Davinopólis', 'Douradinha', 'Eldorado', 'Entre Rios', 'Esperantina', 'Estrela do Norte', 'Fátima', 'Feira Nova', 'Filadélfia', 'Floresta', 'Formoso', 'General Carneiro', 'Gravataí', 'Guaíra', 'Guaraci', 'Guaraciaba', 'Hidrolândia', 'Humaitá', 'Iguatu', 'Inajá', 'Independência', 'Indianápolis', 'Ipueiras', 'Iracema', 'Irati', 'Itabaiana', 'Itajá', 'Itambé', 'Itapeva', 'Itapiranga', 'Itaporanga', 'Jaborandi', 'Jacutinga', 'Jandaíra', 'Japurá', 'Jardim', 'Jardinópolis', 'Jatobá', 'Jundiá', 'Jurema', 'Jussara', 'Lagoa Grande', 'Lagoa Santa', 'Lajeado', 'Laranjal', 'Maravilha', 'Massaranduba', 'Mesquita', 'Milagres', 'Mirador', 'Monte Alegre', 'Monte Castelo', 'Morrinhos', 'Mulungu', 'Mundo Novo', 'Natividade', 'Nazaré', 'Nova Aurora', 'Nova Fátima', 'Nova Olímpia', 'Nova Olinda', 'Nova Santa Rita', 'Nova União', 'Nova Veneza', 'Novo Horizonte', 'Novo Santo Antônio', 'Ouro Branco', 'Ouro Verde', 'Pacatuba', 'Palestina', 'Palmas', 'Palmeira', 'Palmital', 'Paraíso', 'Parnamirim', 'Passagem', 'Pau D’Arco', 'Paulista', 'Pedra Branca', 'Pedra Preta', 'Petrolândia', 'Pilar', 'Pilões', 'Pinhalzinho', 'Pinhão', 'Pitangueiras', 'Planaltina', 'Planalto', 'Praia Grande', 'Prata', 'Presidente Bernardes', 'Presidente Dutra', 'Presidente Juscelino', 'Presidente Kennedy', 'Presidente Médici', 'Primavera', 'Queimadas', 'Redenção', 'Riachão', 'Riachinho', 'Riacho de Santana', 'Riachuelo', 'Rio Branco', 'Rio Claro', 'Rio Negro', 'Rio Verde', 'Ruy Barbosa', 'Salgadinho', 'Saltinho', 'Santa Bárbara', 'Santa Cecília', 'Santa Cruz', 'Santa Filomena', 'Santa Helena', 'Santa Inês', 'Santa Isabel', 'Santa Lúcia', 'Santa Luzia', 'Santa Maria', 'Santa Rita', 'Santa Rosa de Lima', 'Santa Teresinha', 'Santa Terezinha', 'Santana', 'Santarém', 'Santo André', 'São Bento', 'São Carlos', 'São Domingos', 'São Francisco', 'São Francisco de Paula', 'São Gabriel', 'São Gonçalo do Amarante', 'São João', 'São João Batista', 'São João do Paraíso', 'São José do Divino', 'São Martinho', 'São Pedro', 'São Sebastião', 'São Simão', 'São Tomé', 'São Vicente', 'São Vicente Ferrer', 'Sapucaia', 'Sarandi', 'Serrinha', 'Sertãozinho', 'Sítio Novo', 'Sobradinho', 'Soledade', 'Tabatinga', 'Tangará', 'Tapejara', 'Tapira', 'Tapiraí', 'Tavares', 'Teodoro Sampaio', 'Terra Nova', 'Terra Roxa', 'Toledo', 'Trindade', 'Triunfo', 'Turmalina', 'Turvo', 'Valença', 'Vargem', 'Vargem Bonita', 'Várzea', 'Várzea Grande', 'Vera Cruz', 'Viana', 'Viçosa', 'Wenceslau Braz', 'São Paulo', 'Rio de Janeiro']
#a = process.extract(word_to_check,result,limit=3)
#res = str(fillVName(word_to_check))
#print(res)
#dif.get_close_matches(res, ambiguas, n=1, cutoff=0.7)
#print(resamb)

In [ ]:
res = str(fillVName('Vicosa MG')) 
df3 = pd.DataFrame(columns=('ID', '%Similaridade', 'Local_Sugerido','Ambiguidade'))
dif.get_close_matches(res, ambiguas, n=1, cutoff=1)
if not dif.get_close_matches(res, ambiguas, n=1, cutoff=1):
    df3.loc[i] = ['0', str(fillVmatch(res)), res,'N']
else:
    df3.loc[i] = ['0', str(fillVmatch(res)), res,'S']
df3

In [ ]:
orig = 'viçosa mg'
orig2 = list(orig)
seq_matcher = dif.SequenceMatcher(None, orig2, res)
for operation, i1,i2,j1,j2 in seq_matcher.get_opcodes():
    if operation == "delete":
        print("Deleting Sequence : '{}' from l1".format(orig[i1:i2]))
        orig2[i1:i2] = [""] * len(orig[i1:i2])
    #elif operation == "replace":
     #   print("Replacing Sequence : '{}' in l1 with '{}' in l2".format(orig[i1:i2], res[j1:j2]))
     #   orig2[i1:i2] = [""] * len(orig[i1:i2])
     #   orig2.insert(i1, res[j1:j2])
    #elif operation == "insert":
     #   print("Inserting Sequence : '{}' from l2 at {} in l1".format(res[j1:j2], i1))
      #  orig2.insert(i1, res[j1:j2])
    elif operation == "equal":
        print("Equal Sequences. '{}' No Action Needed.".format(orig[i1:i2]))

print("\nFinal Sequence : {}".format("".join(orig2)))

In [ ]:
print("Deleting Sequence : '{}' from l1".format(orig[i1:i2]))

In [ ]:
dicionario = dic[0]
dicionario2 = {}
index = 0
limit = 50
#df2 = pd.DataFrame(columns=('ID', '%Similaridade', 'Local_Sugerido','Ambiguidade'))
for i in dicionario:
   index += 1
   if index == limit:
        break
   print(str(fillVName(re.sub('\W+',' ', dicionario[i]))))
   print(str(fillVmatch(dicionario[i])))
   print(str(re.sub('\W+',' ', dicionario[i])))

recife
100
recife
icó
90
méxico
brasília
90
brasília df

0
guadalajara jalisco 
icó
90
ciudad de méxico
nova brasilândia d'oeste
90
brasil
são paulo
100
são paulo
icó
90
méxico
rio de janeiro
100
rio de janeiro
horizonte
90
belo horizonte minas gerais
0
0

brasília
90
brasil brasília

0
gama df brazil
teresina
95
teresina pi
açu
90
são paulo brasil
0
0

barra do garças
95
barra do garças mt
uauá
90
alcaldía cuauhtemoc
nova brasilândia d'oeste
90
brasil
macaé
100
macaé
nova brasilândia d'oeste
90
brasil
0
0

barra do garças
95
barra do garças mt
brasília
95
brasília df
brasília
90
brasília df

0
entre vênus e marte aprox 
0
0


0
a pale blue dot in space
são josé dos campos
95
são josé dos campos sp
nova brasilândia d'oeste
90
brasil

0
dublin city ireland
nova brasilândia d'oeste
90
brasil
barra do garças
95
barra do garças mt
teresina
95
teresina pi
manaus
90
manaus brasilien
0
0

0
0

ceará
90
ceará brazil
florianópolis
90
florianópolis brasil
uraí
90
gs zz9 plural z α earth
apucaran